In [3]:
#разбиваем всю БД каждые 10000 строк на разделы lim 
import pymysql.cursors
import pandas as pd
import numpy as np

import re
import emoji

import pymorphy2
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))

# препроцессинг

def preprocess_text(text):
    def replace_three_or_more(text):
        pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
        return pattern.sub(r"\1\1", text)

    def emo_to_text(text):  
        emoticon = {'улыбка шутка' : ['\:\)', '\:\-\)', '\:\=\)'],
                    'грусть разочарование' : ['\:\(', '\:\-\(', '\:\=\('],
                    'открытая радость' : ['\=\)', '\=\-\)'],
                    'усмешка хихиканье' : ['\:>', '\:\->', '\:\=>'],
                    'улыбка' : ['\:\]', '\:\-\]', '\:\=\]'],
                    'смех' : ['\:D', '\:\-D', '\:\=D'],
                    'сильный смех' : ['\:DD', '\:\-DD', '\:\=DD'],
                    'сарказм' : ['\:\}', '\:\-\}', '\:\=\}'],
                    'подмигивание заигрывание' : ['\;\)', '\;\-\)', '\;\=\)'],
                    'молчание' : ['\:Х', '\:\-Х', '\:\=Х'],
                    'удивленное разочарование' : ['8\(', '8\-\(', '8\=\('],
                    'изумление' : ['B\-о', 'B\=о'],
                    'радостное удивление' : ['\%\)'],
                    'зевота' : ['\|\-o', '\|\=o'],
                    'смущение' : ['\:S', '\:-S', '\:\=S'],
                    'равнодушие скука' : ['\:\|', '\:\-\|', '\:\=\|'],
                    'недоверие сомнение' : ['\:\/', '\:\-\/', '\:\=\/'],
                    'дразнящее показывание языка' : ['\:ь', '\:\-ь', '\:\=ь'],
                    'поцелуй' : ['\:\*', '\:\^\*', '\:\-Ф'],
                    'шалость дурачество' : ['\:0\)'],
                    'кривая ухмылка' : ['\:7', '\:\-7']}
                    
        for key, value in emoticon.items():
          text = re.sub(r"|".join(value), ' ' + key, text)
        return text

    try:
      text = emo_to_text(text)
      text = emoji.demojize(text, delimiters=("", "")) #преобразует эмодзи-символы в слова
      text = text.lower().replace("ё", "е")
      text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
      text = re.sub('@[^\s]+', 'USER', text)
      text = re.sub('[^a-zA-Zа-яА-Я]+', ' ', text) #удаляем цифры и спецсимволы
      text = replace_three_or_more(text) #удаляет дублирование букв, если число букв более 3
      text = re.sub(' +', '|', text)
      return text.strip()
    except:
      print(f'Слово {text} не прошло препроцессинг')
      return ''

# Connect to the database
def get_connection():
    connection = pymysql.connect(host='database-3.chim8btj05zt.us-east-1.rds.amazonaws.com',
                                 user='admin',
                                 password='01035007',
                                 db='test1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection


connection = get_connection()

morph = pymorphy2.MorphAnalyzer() #по умолчанию русский язык
stemmer = SnowballStemmer("english")

LIM = 1000 #число проходов по БД
try:
    with connection.cursor() as cursor:
        s = "SELECT min(id), max(id) FROM sentiment"
        cursor.execute(s)
        start, stop = cursor.fetchone().values()
        start = 441644379397451776 + 1
        delta = (stop - start) // LIM #интервал id, в котором искать значения в каждом проходе

        #cursor.execute("DROP TABLE IF EXISTS sentiment_v3")
        #connection.commit()
        cursor.execute("""CREATE TABLE IF NOT EXISTS sentiment_v3(
              id bigint(32) NOT NULL PRIMARY KEY,
              tdate varchar(128) DEFAULT NULL,
              tname varchar(128) DEFAULT NULL,
              ttext_stem varchar(256) DEFAULT NULL,
              ttext_prep varchar(256) DEFAULT NULL,
              ttext varchar(256) DEFAULT NULL,
              ttype int(10) DEFAULT '0',
              trep int(10) DEFAULT '0',
              trtw int(10) DEFAULT '0',
              tfav int(10) DEFAULT '0',
              tstcount int(10) DEFAULT '0',
              tfoll int(10) DEFAULT '0',
              tfrien int(10) DEFAULT '0',
              listcount int(10) DEFAULT '0') DEFAULT CHARSET=utf8mb4""")
        connection.commit()

        while stop > start:
          try:
            s = "SELECT * FROM sentiment WHERE id >= %s AND id < %s"
            cursor.execute(s, (start, start + delta))
            start = start + delta
            result = cursor.fetchall()
            raw_data = pd.DataFrame(result, index = None)
            
            print(f"В датафрейм загружены прецеденты в количестве {raw_data.shape[0]}")

            #препроцессинг
            if raw_data.shape[0] > 0:
              print(f"Первая строка: {raw_data.head(1)}")
              raw_data['ttext_prep'] = raw_data['ttext'].apply(preprocess_text)

            else:
              continue
                                   

            #стемминг
            ttext_stem = list()
            for row in raw_data.itertuples():
                words = row.ttext_prep.split('|')
                nf_words = list()
                for word in words:
                  try:
                    #word = word.encode('utf8mb4')
                    ru_word = re.sub('[^а-яА-Я]+', ' ', word)
                    en_word = re.sub('[^a-zA-Z]+', ' ', word)
                    if ru_word == word and word != '': # кириллица
                      parse_word = morph.parse(word)[0]
                      nf_word = parse_word.normal_form
                      nf_words.append(nf_word.strip())
                    elif en_word == word and word != '': # латиница
                      nf_word = stemmer.stem(word)
                      nf_words.append(nf_word.strip())
                    else:
                      pass #смешанные слова не анализируются
                  except:
                    print(f'Слово не преобразовано') 
                    continue
                nf_words = [w for w in nf_words if not w in stops]
                nf_words = '|'.join(nf_words) #ttext_stem
                ttext_stem.append(nf_words)

            if ttext_stem != None:
                ttext_stem = pd.Series(ttext_stem) 
                raw_data['ttext_stem'] = ttext_stem  
                s =  "%s, " * 13 + "%s)"
                insert = "INSERT INTO sentiment_v3 VALUES (" + s    
                for row in raw_data.itertuples():
                    cursor.execute(insert, (row.id, row.tdate,
                                            row.tname, row.ttext_stem, row.ttext_prep, row.ttext,
                                            row.ttype, row.trep,
                                            row.trtw, row.tfav,
                                            row.tstcount, row.tfoll,
                                            row.tfrien, row.listcount))
                connection.commit()
          except Exception as e:
            print(f"Ошибка итерации while. {e}")
            continue
        print("Конвертация завершена")
        
finally:
    connection.close()



Конвертация завершена
